In [ ]:
from shiny import App, render, ui
import shinyswatch
import requests
from bs4 import BeautifulSoup
from googleapiclient.discovery import build
import googlemaps
import tweepy
from transformers import pipeline
import spacy
import os
import google.generativeai as genai
import re 
from spacytextblob.spacytextblob import SpacyTextBlob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
from dotenv import load_dotenv

e:\Users\1167486\AppData\Local\anaconda3\envs\Isis_WebScrapper\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [ ]:
url_test = ''

In [ ]:
def wikipedia_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lanza una excepción para códigos de error HTTP
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = "\n".join([p.get_text() for p in paragraphs])
        return text[:500] + "..." if len(text) > 500 else text # Mostrar un fragmento por ahora
    except requests.exceptions.RequestException as e:
        return f"Error al acceder a Wikipedia: {e}"


In [ ]:
def sentiment_output(input_text):
    nlp_sentiment = None
    try: 
        nlp_sentiment = spacy.load("es_core_news_md")
        nlp_sentiment.add_pipe('spacytextblob')
    except Exception as e:  
        print(f"Error al cargar el modelo de Spacy: {e}")
        return ui.output_text("sentiment_error", f"Error al cargar el modelo de Spacy: {e}")
    all_text_to_analyze ={
        #"Comentarios de YouTube": input.youtube_comments(),
        #"Comentarios de Google Maps": input.maps_comments(),
        #"Posts de Twitter": input.twitter_posts(),
        "Texto de Wikipedia": input_text            
    }
    print(f"Texto a analizar en sentimientos: {all_text_to_analyze}")
    sentiment_data = []
    for source, text in all_text_to_analyze.items():
        if text and not text.startswith("Error"):
            try:
                doc = nlp_sentiment(text)
                polarity = doc._.blob.polarity  
                sentiment='Neutral'
                if polarity > 0.1:
                    sentiment = 'Positivo'
                elif polarity < -0.1:
                    sentiment = 'Negativo'
                sentiment_data.append({"Source": source, "Sentiment": sentiment})
            except Exception as e:
                print(f"Error analyzing sentiment for {source}: {e}")
        else:
            print(f"No text available for sentiment analysis for {source}")
    if not sentiment_data:
        return ui.output_text("No hay texto para analizar.")
        
    df_sentiment= pd.DataFrame(sentiment_data)
    sentiment_counts= df_sentiment.groupby('Source')['Sentiment'].value_counts(normalize=True).unstack().fillna(0)
    fig, ax = plt.subplots(figsize=(10, 6))
    sentiment_counts.plot(kind='bar', stacked=True, ax=ax)
    plt.title('Distribución de Sentimientos')
    plt.xlabel("Source")
    plt.ylabel("Percentage")
    plt.xticks(rotation=45, ha="right")

    for p in ax.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        ax.annotate(f'{height:.1%}', (x + width/2, y + height/2), ha='center', va='center')

    plt.tight_layout()
    return fig
